In [1]:
from mysql.connector import connect, Error
from flask import Flask, request, jsonify
from werkzeug.security import check_password_hash

In [2]:
#Connection
def get_db_connection():
    try:
        connection = connect(
            host="localhost",
            user="root",
            password="password",
            database="Offers2Day"
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None


In [3]:
connection = get_db_connection()

In [14]:
app = Flask(__name__) #Flask app

In [4]:
#Test inputs

email = "yassin"
password = "123"

In [16]:
@app.route('/test', methods=['GET'])
def test():
    return jsonify('hello')

In [17]:
@app.route('/login', methods=['POST'])#login route
def login():
    #User input from the http request
    data = request.get_json() #received the json object from flutter
    email = data.get('email') #user email or username
    password = data.get('password') #user password
    customer_business = "Customer" if data.get("isCustomer") == "True" else "Business" #This will determine whether the user is a customer or a business

    connection = get_db_connection()

    if connection:
        cursor = connection.cursor(dictionary=True)#data will get returned as a dict not a tuple

        #our query
        #customer
        if customer_business == "Customer":
            query = f"SELECT * FROM Customer C, CustomerPasswords CP WHERE C.email = {email} OR C.username = {email} AND C.userID = CP.userID" 
        #business
        else:
            query = f"SELECT * FROM Businesses B, BusinessPasswords BP WHERE B.businessEmail = {email} OR B.businessName = {email} AND B.businessID = BP.businessID" 
        
        #Execute the query
        cursor.execute(query)
        #Get the first result
        user = cursor.fetchone()#dict

        #check_password_hash(user['password'], password):#check_password_hash is a function that hashes the user password before checking the db
        if user and user['password'] == password:
            cursor.close()
            connection.close()
            return jsonify({'message': 'Login successful'}), 200
        else:
            cursor.close()
            connection.close()
            return jsonify({'message': 'Login failed'}), 401

In [18]:
def signup():
    '''signup

    The function that gets the inputed info from the user and
    places it into the database.
    '''

    data = request.get_json() #received the json object from flutter

    connection = get_db_connection()    

    if connection:
        pass
    else:
        pass

In [5]:
q = f"SELECT * FROM Customers C, CustomerPasswords CP WHERE C.userID = CP.userID AND (C.email = {email} OR C.username = {email}) AND CP.password = {password}"
cursor = connection.cursor()
cursor.execute(q)
cursor.fetchall()

ProgrammingError: 1054 (42S22): Unknown column 'yassin' in 'where clause'